<a href="https://colab.research.google.com/github/3x10e8/hrir/blob/main/hrir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import CIPIC filterbank

In [1]:
# clone repo with updated .mat file
! git clone https://github.com/3x10e8/hrir.git

fatal: destination path 'hrir' already exists and is not an empty directory.


In [2]:
from scipy.io import loadmat
# CIPIC database: https://www.ece.ucdavis.edu/cipic/ 
# hrir_final.mat taken from https://github.com/marekjg/hrtf/tree/master/cipic_hrir
cipic = loadmat('hrir/hrir_final_R2015b.mat')

In [3]:
# Install pydub for converting mp3 to np array
try:
  import pydub
except ImportError:
  !pip install pydub
  import pydub

from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt

# from https://stackoverflow.com/questions/53633177/how-to-read-a-mp3-audio-file-into-a-numpy-array-save-a-numpy-array-to-mp3
def mp3read(f, normalized=False):
    """MP3 to numpy array"""
    a = pydub.AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

## Import audio file

In [4]:
# Included audio sample is (c) Old School Tie
FILENAME = 'hrir/sample.mp3'
fs, y = mp3read(FILENAME, normalized = True)
y = y.transpose() # Shape=(NCHAN, NSAMPLES)

## Trim audio file

In [5]:
tstart_s = 106.6
tend_s = tstart_s + 8 # play a 5 second clip
y_clip = y[:, int(tstart_s*fs) :int(tend_s*fs)]

# fade in
y_clip[:, :fs] *= np.arange(start=0, stop=1, step=1/fs)

# fade out
y_clip[:, -fs::] *= np.arange(start=1, stop=0, step=-1/fs)

Audio(data=y_clip, rate=fs, autoplay=True)

## Filter audio sample through L/R HRIRs

In [13]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import IPython.display as ipd 

def filter(az = 0, el = 0):
  firL = cipic['hrir_l'][az][el]
  firR = cipic['hrir_r'][az][el]

  plt.plot(firL, label='Left HRIR')
  plt.plot(firR, label='Right HRIR')
  plt.xlabel('tap#')
  plt.title('Selected HRIR Filters for Left/Right Channels')

  outL = np.convolve(y_clip[0,:], firL)
  outR = np.convolve(y_clip[1,:], firR)
  out = np.stack([outL, outR], axis=0)

  ipd.display(ipd.Audio(data=out, rate=fs, autoplay=True))

  if 0:
    # Visualize the results as XY plots of the signal before/after processing
    NUM_SAMPLES = fs
    NTAPS = len(firL)
    plt.subplot(121)
    plt.plot(y_clip[0][:NUM_SAMPLES], 
            outL[NTAPS :NTAPS+NUM_SAMPLES], 
            label='left')
    plt.plot(y_clip[1][:NUM_SAMPLES], 
            outR[NTAPS :NTAPS+NUM_SAMPLES], 
            label='right')
    plt.xlabel('Input')
    plt.ylabel('HRIR Output')
    plt.legend()
    plt.axis('equal')

    plt.subplot(122)
    plt.plot(y_clip[0][:NUM_SAMPLES], 
            y_clip[0][:NUM_SAMPLES],
            label='left')
    plt.plot(y_clip[1][:NUM_SAMPLES], 
            y_clip[1][:NUM_SAMPLES], 
            label='right')

az_slider = widgets.IntSlider(
    value=5,
    min=0, 
    max=len(cipic['hrir_l'])-1,
)

el_slider = widgets.IntSlider(
    value=40,
    min = 0,
    max = len(cipic['hrir_l'][0])-1,
    orientation = 'vertical'   
)

interact(filter, 
         az=az_slider, #(0, len(cipic['hrir_l']-1)), 
         el=el_slider, #(0, len(cipic['hrir_l'][0]-1)),
         continuous_update=False,
)

interactive(children=(IntSlider(value=5, description='az', max=24), IntSlider(value=40, description='el', max=…

<function __main__.filter(az=0, el=0)>